Copyright (c) 2008 Idiap Research Institute, http://www.idiap.ch/
    
Written by Suhan Shetty <suhan.shetty@idiap.ch>,


In [1]:
%load_ext autoreload
%autoreload 2
from panda_kinematics import PandaKinematics

import torch
import numpy as np
np.set_printoptions(2, suppress=True)
torch.set_printoptions(2, sci_mode=False)

import sys
sys.path.append('../')
from panda_cost_utils import SDF_Cost, PandaCost
from ttgo import TTGO
from utils import test_ttgo
device = 'cpu'#torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cpu


In [2]:
# Hyper-params
d0_theta = 50
dh_x = 0.005
margin = 0
kr = 5
b_goal = 0.05 ; b_obst= 0.01; b_orient=0.2;
nswp=20; rmax=500; kr=2;
d_type = "uniform"

### Define the cost/pdf function

In [3]:
data_sdf = np.load('./data/sdf.npy', allow_pickle=True)[()]
sdf_matr = data_sdf['sdf_matr']  
bounds = torch.tensor(data_sdf['bounds']).to(device) # Bound of the environment
sdf_tensor = torch.from_numpy(sdf_matr).to(device)
sdf_cost = SDF_Cost(sdf_tensor=sdf_tensor, domain=bounds, device=device)
env_bound = data_sdf['env_bound']  
shelf_bound = data_sdf['shelf_bound'] 
box_bound = data_sdf['box_bound'] 

# key-points on the body of the robot for collision check
data_keys = np.load('./data/sphere_setting.npy', allow_pickle=True)[()]# key_points
status_array = data_keys['status_array']
body_radius = data_keys['body_radius']
relative_pos = data_keys['relative_pos']
key_points_weight = torch.from_numpy(status_array).float().to(device) >0 # 8xMx1
key_points_weight[-1] = 1*margin
key_points_margin = torch.from_numpy(body_radius).float().to(device)#
key_points_pos = torch.from_numpy(relative_pos).float().to(device)
key_points = [key_points_pos, key_points_weight, key_points_margin]

# define the robot
panda = PandaKinematics(device=device, key_points_data=key_points)

############################################################

# Define the cost function

# Specify the doesired orientation
Rd_0 = torch.tensor([[ 0.7071,0.7071,0.], [0.,0.,1],[0.7071, -0.7071, 0.]]).to(device) # desired orientation
v_d = torch.tensor([0.,0.,1.]).to(device)
# Rd = torch.tensor([[ 0,0.,0.], [0.,0.,1],[0., 0., 0.]])

pandaCost = PandaCost(robot=panda, sdf_cost=sdf_cost,
                    Rd_0=Rd_0, v_d=v_d,b_obst=b_obst, 
                    b_goal=b_goal,b_orient=b_orient,device=device)  


def cost(x): # For inverse kinematics
    return pandaCost.cost_ik(x)[:,0]

def cost_all(x): # For inverse kinematics
    return pandaCost.cost_ik(x)

def pdf(x):
    x = x.to(device)
    pdf_ = torch.exp(-cost(x)**2) 
    return pdf_

############################################################################

# Define the domain for discretization

n_joints=7
d_theta_all = [d0_theta]*n_joints
d_theta = [int(d_theta_all[joint]) for joint in range(n_joints)]

# type of discretization of intervals of decision variables
if d_type == 'uniform':
    domain_decision = [torch.linspace(panda.theta_min[i],panda.theta_max[i],d_theta[i]).to(device) for i in range(n_joints)]
else: # logarithmic scaling
    domain_decision = [exp_space(panda.theta_min[i],panda.theta_max[i],d_theta[i]).to(device) for i in range(n_joints)]

# task space of the manipulator (the shelf)
env_bounds = torch.from_numpy(shelf_bound)
x_min = env_bounds[:,0]
x_max = env_bounds[:,1]
x_max[0] = 0.75; x_min[0]=0.45
x_max[1] = x_max[1]-0.1
x_min[1] = x_min[1]+0.1
x_max[-1] = 0.75; x_min[-1] = 0.


domain_task = [torch.linspace(x_min[i], x_max[i], int((x_max[i]-x_min[i])/dh_x)) for i in range(3)]
domain = domain_task + domain_decision
print("Discretization: ",[len(x) for x in domain])

#######################################################################################
# Fit the TT-Model
# with torch.no_grad():
ttgo = TTGO(domain=domain,pdf=pdf, cost=cost, device=device)
ttgo.cross_approximate(rmax=rmax, nswp=nswp, kickrank=kr)
ttgo.round(eps=1e-3)

###########################################################
# Prepare for the task
sites_task = list(range(len(domain_task)))
ttgo.set_sites(sites_task)
############################################################ 
print("############################")
print("Test the model")
print("############################")

# generate test set
ns = 100
test_task = torch.zeros(ns,len(domain_task)).to(device)
for i in range(len(domain_task)):
    unif = torch.distributions.uniform.Uniform(low=domain_task[i][0],high=domain_task[i][-1])
    test_task[:,i]= torch.tensor([unif.sample() for i in range(ns)])

file_name = 'panda_ik.pickle'
torch.save({
    'tt_model':ttgo.tt_model,
    'panda': panda,
    'pandaCost':pandaCost,
    'sdf_cost':sdf_cost,
    'w': (pandaCost.w_goal,pandaCost.w_obst,pandaCost.w_orient),
    'b': (b_goal,b_obst,b_orient),
    'margin': margin,
    'key_points_weight':key_points_weight,
    'key_points_margin':key_points_margin,
    'domains': domain,  
    'Rd_0': Rd_0,
    'v_d':v_d,
    'test_task': test_task,
}, file_name)




Discretization:  [60, 151, 150, 50, 50, 50, 50, 50, 50, 50]
cross device is cpu
Cross-approximation over a 10D domain containing 1.06172e+18 grid points:
Note: The algorithm converges as the ratio tt-new-norm/tt-old-norm settles to 1. For TTGO, the convergence is not important, just keep iterating as long as the ratio > 1
iter: 0  | tt-new-norm/tt-old-norm: 2.524e-05 | time:   0.0936 | largest rank:   1
iter: 1  | tt-new-norm/tt-old-norm: 1.423e+00 | time:   0.3083 | largest rank:   3
iter: 2  | tt-new-norm/tt-old-norm: 1.312e+00 | time:   0.5487 | largest rank:   5
iter: 3  | tt-new-norm/tt-old-norm: 1.694e+00 | time:   0.9096 | largest rank:   7
iter: 4  | tt-new-norm/tt-old-norm: 3.697e+00 | time:   1.4061 | largest rank:   9
iter: 5  | tt-new-norm/tt-old-norm: 1.506e+00 | time:   2.1659 | largest rank:  11
iter: 6  | tt-new-norm/tt-old-norm: 1.254e+00 | time:   3.1952 | largest rank:  13
iter: 7  | tt-new-norm/tt-old-norm: 9.779e-01 | time:   4.4443 | largest rank:  15
iter: 8  | t

In [4]:
# generate test set
test_task = torch.zeros(1000,len(domain_task)).to(device)
for i in range(len(domain_task)):
    unif = torch.distributions.uniform.Uniform(low=domain_task[i][0],high=domain_task[i][-1])
    test_task[:,i]= torch.tensor([unif.sample() for i in range(1000)])

ns = 100
test_task  = test_task[(sdf_cost.sdf(test_task)-0.09)>0][:ns] # 

joint_angles = torch.zeros((test_task.shape[0],7))
x_tasks = torch.zeros((test_task.shape[0],3))
for i in range(test_task.shape[0]):
    x_task =test_task[i].view(1,-1)
    samples,_ = ttgo.sample(n_samples=100,alpha=0.9,x_task=x_task)
    best_sample = ttgo.choose_best_sample(samples)
    sol,_ = ttgo.optimize(best_sample)
    joint_angles[i] = sol[:,3:]
    x_tasks[i] = sol[:,:3]
    


In [5]:
# torch.save({'task':x_tasks,'theta':joint_angles},"ik_data.pickle",)

In [6]:
.

SyntaxError: invalid syntax (<ipython-input-6-a5d5b61aa8a6>, line 1)

## Visualization

In [1]:
import pybullet_data
from panda_visualization_utils import *
import pybullet as p
from functools import partial
# import the environment (SDF and for graphics visualization in pybullet)
import sys
DATA_PATH = './data'
body_sphere_path = './data/sphere_setting.npy'
sdf_path = './data/sdf.npy'
urdf_path = './data/urdf/frankaemika_new/panda_arm.urdf'

p.connect(p.GUI)
data = np.load(sdf_path, allow_pickle=True)[()]
sdf_matr = data['sdf_matr']  #SDF tensor
obstacles = data['obstacles'] #obstacles parameters
colors = [[0.8, 0.5, 0.5, 1]]*len(obstacles)
obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet (np.zeros(3), np.zeros(3), obstacles, colors=colors)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
robot_urdf ='./data/urdf/frankaemika_new/panda_arm.urdf'
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])
set_q_std = partial(set_q,robot_id, pb_joint_indices)
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()
pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand_joint')
alpha = np.deg2rad(52)
quat = p.getQuaternionFromAxisAngle((0,0,1),alpha)
p.resetBasePositionAndOrientation(robot_id, (0,0,0.05), quat)

In [2]:
s = np.random.randint(0,test_task.shape[0]-1)
sample_xe =test_task[s] # torch.tensor([0.58, 0.68, 0.78])
print(sample_xe)
# torch.tensor([0.4435, 0.4347, 0.2139]) # bottom center
# torch.tensor([0.4991, 0.6301, 0.3695]) # bottom left
# torch.tensor([0.6352, 0.4708, 0.7535]) # top left
# torch.tensor([0.6789, 0.1950, 0.6976]) # top shelf center
# torch.tensor([0.6635, 0.6250, 0.2031]) # bottom, in, inaccessible
# tirch.tensor([0.4228, 0.4441, 0.8760]) # top 
# sample_xe = torch.tensor([0.8, -0.1, 0.3]) #torch.tensor([0.8, -0.3, 0.35])#torch.torch.tensor([ 0.6404, 0.2350,  0.549])#torch.tensor([ 0.7130, -0.3007, -0.1276]) #
sample_xe[0] = 0.5
n_solutions= 2
n_samples_tt = 200*n_solutions
n_samples_rand= 10*n_samples_tt

alpha=0.5; norm=1 ;

t1 = time.time()
samples_tt, samples_idx = ttgo.sample(n_samples=n_samples_tt, x_task=sample_xe.reshape(1,-1),alpha=alpha, norm=norm)
state_k_tt = ttgo.choose_top_k_sample(samples_tt,n_solutions)

#Optimize
state_k_tt_opt = 1*state_k_tt
for i, state in enumerate(state_k_tt):
    state_k_tt_opt[i,:],_= ttgo.optimize(state,bound=True)
t2 = time.time()

             
c_tt =  cost(state_k_tt_opt)

print("Cost-mean-tt:",torch.mean(c_tt,dim=0))


NameError: name 'test_task' is not defined

In [ ]:
# _,_,test_sphere = create_primitives(p.GEOM_SPHERE, radius = 0.02)
# p.resetBasePositionAndOrientation(test_sphere, (0,0,1.), (0,0,0,1))

x_target = sample_xe[:3].numpy()
joint_angles_k = state_k_tt[:,3:].numpy() 
joint_angles_k_opt = state_k_tt_opt[:,3:].numpy() 


_ , _,sphere_id = create_primitives(p.GEOM_SPHERE, radius = 0.02)
pos = x_target[:]

p.resetBasePositionAndOrientation(sphere_id, pos, (0,0,0,1))


k = joint_angles_k.shape[0]
dt = 1
dT = 2
for i in range(2*k):
    set_q_std(joint_angles_k[i%k])
    time.sleep(dt)
    set_q_std(joint_angles_k_opt[i%k])
    time.sleep(2*dt)
    